In [1]:
import os
import sys
sys.path.append("../Train_utils")
sys.path.append('../Models')
sys.path.append("../Dataset_utils")
import numpy as np

In [2]:
from TT_class import trainer

In [3]:
#Dataset
from DataLoader import UAV_GPS_Dataset
DS=UAV_GPS_Dataset(
    data_dir="../../Datasets/GPS/train_sorted",
    representations=["rgb","poses"]
)

In [4]:
#Model
from torch import nn
import torch
from Encoding_Decoding_modules.Basic_Encoding_Decoding_Module import ResNET_Convolutional_EDM
from DL_utils.utils import NeuralNet
from DL_utils.ResNET import ResNET_ENC
from DL_utils.Graph_layers import Conv_WL_GNN,KNN_graph,image_batch_to_graph_batch,split_batch_graph
from DL_utils.Graph_modules import WL_GNN_ENC
from CNN_graph import ResNET_WL

CWLGNN=Conv_WL_GNN(225,5)

EDM=ResNET_ENC(
    repr_sizes=[[3,4,5],[5,6,7]],
    kernel_sizes=[[3,5],[5,5]],
    bridge_kernel_size=3,
    act=nn.ReLU(),
    bridge_act=nn.ReLU(),
    lay_act=nn.ReLU(),
    batch_norm=True,
    dropout=None,
    stride=[[1,1],[1,2]],
    )
RWL=ResNET_WL(
    image_encoding_mode=EDM,
    graph_encoding_mode=WL_GNN_ENC([86400,100,50,12]),
    Estimation_NeuralNet=NeuralNet(layer_sizes=[84,12],activators=nn.LeakyReLU(),batch_norm=True,dropout=None)
    )
RWL.cuda()

ResNET_WL(
  (MSEloss): MSELoss()
  (img_enc): ResNET_ENC(
    (im_layers): ModuleList(
      (0): set_ResNET_conv(
        (shortcut): ModuleList(
          (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (pre_layer): ModuleList(
          (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(4, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (4): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (lay_act): ReLU()
      )
      (1): set_ResNET_conv(
        (shortcut): ModuleList(
          (0): Conv2d(5, 7, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=T

In [5]:
#RWL.E_NN.layers[0][0].layer[0]
#RWL.E_NN.is_cuda()
next(RWL.parameters()).device

device(type='cuda', index=0)

In [5]:
tr=trainer(
    model=RWL,
    dataset=DS,
    epochs=100,
    folds=5,
    batch_size=16,
    use_cuda=True,
    data_dir="../../Results/Debug",
    in_device=None,
    num_workers=0,
    args=["rgb","poses"],
    uniform=True
)
tr.optimizer=torch.optim.Adam(RWL.parameters(),lr=0.001)

In [6]:
PRS=torch.utils.data.Subset(DS, np.arange(32))

DL=torch.utils.data.DataLoader(PRS,batch_size=16,shuffle=True,num_workers=0,drop_last=True)

In [7]:
tr.train(DL)

instances: 1it [00:02,  2.93s/it]

data:	regression_loss 3378736791552.0000	total_loss 3378736791552.0000 	batch 16.0000


instances: 2it [00:05,  2.95s/it]

data:	regression_loss 3378737053696.0000	total_loss 3378737053696.0000 	batch 16.0000


{'train': {'regression_loss': [3378736791552.0, 3378737053696.0],
  'total_loss': [3378736791552.0, 3378737053696.0]},
 'test': {'regression_loss': [], 'total_loss': []}}

In [17]:
tr.test(DL)

Test: 1it [00:03,  3.34s/it]

{'total_loss': tensor(3.3787e+12, grad_fn=<AddBackward0>), 'regression_loss': tensor(3.3787e+12, grad_fn=<MseLossBackward0>)}
data:	total_loss 3378732335104.0000	regression_loss 3378732335104.0000 	batch 16.0000


Test: 2it [00:06,  3.36s/it]

{'total_loss': tensor(3.3787e+12, grad_fn=<AddBackward0>), 'regression_loss': tensor(3.3787e+12, grad_fn=<MseLossBackward0>)}
data:	total_loss 3378740461568.0000	regression_loss 3378740461568.0000 	batch 16.0000


{'train': {'total_loss': [3378743869440.0, 3378729189376.0],
  'regression_loss': [3378743869440.0, 3378729189376.0]},
 'test': {'total_loss': [3378732335104.0, 3378740461568.0],
  'regression_loss': [3378732335104.0, 3378740461568.0]}}

In [18]:
tr.train_test(PRS,PRS)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]
                                              
Epoch:   0%|          | 0/100 [00:04<?, ?it/s]
instances: 0it [00:04, ?it/s]
instances: 1it [00:04,  4.48s/it]

data:	total_loss 3378733383680.0000	regression_loss 3378733383680.0000 	batch 16.0000


                                              
Epoch:   0%|          | 0/100 [00:08<?, ?it/s]
instances: 1it [00:08,  4.48s/it]
instances: 2it [00:08,  4.46s/it]


data:	total_loss 3378739675136.0000	regression_loss 3378739675136.0000 	batch 16.0000



Test: 0it [00:00, ?it/s]
                                              
Epoch:   0%|          | 0/100 [00:12<?, ?it/s]
Test: 1it [00:03,  3.37s/it]

...

{'total_loss': tensor(3.3787e+12, grad_fn=<AddBackward0>), 'regression_loss': tensor(3.3787e+12, grad_fn=<MseLossBackward0>)}
data:	total_loss 3378742558720.0000	regression_loss 3378742558720.0000 	batch 16.0000



                                                       
Epoch:  99%|█████████▉| 99/100 [27:22<00:16, 16.43s/it]
Test: 2it [00:06,  3.38s/it]
Epoch:  99%|█████████▉| 99/100 [27:22<00:16, 16.43s/it]

{'total_loss': tensor(3.3787e+12, grad_fn=<AddBackward0>), 'regression_loss': tensor(3.3787e+12, grad_fn=<MseLossBackward0>)}
data:	total_loss 3378729713664.0000	regression_loss 3378729713664.0000 	batch 16.0000
epoch 99.00%


Epoch: 100%|██████████| 100/100 [27:23<00:00, 16.43s/it]


OrderedDict([('img_enc.im_layers.0.shortcut.0.weight',
              tensor([[[[ 0.0884, -0.0916, -0.0919],
                        [-0.0423,  0.0047,  0.0728],
                        [-0.0437,  0.0768, -0.1452]],
              
                       [[-0.0679, -0.0312, -0.1507],
                        [ 0.0952,  0.0343, -0.0087],
                        [-0.0784,  0.0486,  0.0172]],
              
                       [[-0.0870,  0.1492, -0.1908],
                        [-0.0127, -0.1370, -0.1265],
                        [-0.1516,  0.1340,  0.0595]]],
              
              
                      [[[-0.1830,  0.1557, -0.0417],
                        [ 0.1283, -0.0556, -0.1328],
                        [-0.0470,  0.0465, -0.1676]],
              
                       [[ 0.0397,  0.0952,  0.1392],
                        [ 0.0925,  0.0211, -0.1611],
                        [ 0.0828,  0.2008,  0.0205]],
              
                       [[-0.1759, -0.1504,  0.1633],
 

In [14]:
EDM.ENC

ResNET_ENC(
  (im_layers): ModuleList(
    (0): set_ResNET_conv(
      (shortcut): ModuleList(
        (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (pre_layer): ModuleList(
        (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(4, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (4): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (lay_act): ReLU()
    )
    (1): set_ResNET_conv(
      (shortcut): ModuleList(
        (0): Conv2d(5, 7, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (pre_layer): ModuleLis

In [11]:
pr=torch.randn([16, 7, 12]).cuda()
print(pr.shape)
print((pr.flatten(1)).shape)

torch.Size([16, 7, 12])
torch.Size([16, 84])


In [14]:
a=torch.randn([16, 7, 12])
a.device

device(type='cpu')

In [25]:
pr.get_device()

0

In [27]:
a.to(pr.device)
a.get_device()

-1